In [3]:
import logging
import fastwer
import numpy as np
import wandb
import torch.multiprocessing
from transformers import RobertaConfig, EncoderDecoderConfig
torch.multiprocessing.set_sharing_strategy('file_system')

import pandas as pd
from simpletransformers.seq2seq import (
    Seq2SeqModel,
    Seq2SeqArgs,
)

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = Seq2SeqArgs()
model_args.num_train_epochs = 10
# model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = False
model_args.tensorboard_dir = "runs"
model_args.max_length = 50
model_args.train_batch_size=10
model_args.overwrite_output_dir=True
model_args.wandb_project = "cs224u"
model_args.use_multiprocessed_decoding = True

config_encoder = RobertaConfig()
# config_decoder = RobertaConfig(is_decoder=True, add_cross_attention=True)
config_decoder = RobertaConfig()
config_decoder.is_decoder = True
config_decoder.add_cross_attention = True
# model_args = {} #{"use_multiprocessed_decoding": True}
config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
# encoder_decoder_name = "roberta"
# model = EncoderDecoderModel(config=config)
# model = Seq2SeqModel(
#     encoder_type="roberta",
#     encoder_name="roberta-base",
#     decoder_name="roberta-base",
#     args=model_args,
#     config=config,
#     use_cuda=True,
# )

# Initialize model
# model = Seq2SeqModel(
#     encoder_decoder_type="bart",
#     encoder_decoder_name="facebook/bart-large",
#     args=model_args,
#     use_cuda=True,
# )

# model = Seq2SeqModel(encoder_decoder_type="bart", encoder_decoder_name="./outputs/best_model", args=model_args, use_cuda=True,)
model = Seq2SeqModel(encoder_type="roberta", encoder_name="./outputs/checkpoint-9702-epoch-6/encoder", decoder_name="./outputs/checkpoint-9702-epoch-6/decoder", args=model_args, config=config, use_cuda=True)

# model = Seq2SeqModel(
#     encoder_type="bert",
#     encoder_name="bert-base-uncased",
#     decoder_name="bert-base-uncased",
#     args=model_args,
#     use_cuda=True,
# )


def count_matches(labels, preds):
    return sum(
        [
            1 if label == pred else 0
            for label, pred in zip(labels, preds)
        ]
    )

def get_wer(labels, preds):
    return np.mean(
        [
            fastwer.score_sent(pred, label)
            for label, pred in zip(labels, preds)
        ]
    )

AttributeError: module 'torch' has no attribute 'cuda'

In [2]:
import pandas as pd

train_df = pd.read_pickle("train.pkl")
train_df = train_df.dropna()
dev_df = pd.read_pickle("dev.pkl")

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [3]:
# train_df = train_df.head(100)

In [4]:
# dev_df = dev_df.head(20)

In [ ]:
# Train the model
wandb.init(project='cs224u', entity='gbanerje')

# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

# Model training here

model.train_model(
    train_df, eval_data=dev_df, matches=count_matches, wer=get_wer, show_running_loss=True, args={'fp16': False}
)

wandb.join()

# # Evaluate the model
results = model.eval_model(dev_df)

wandb: Currently logged in as: gbanerje (use `wandb login --relogin` to force relogin)


INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_utils: Saving features into cached file cache_dir/._outputs_checkpoint-9702-epoch-6_encoder-._outputs_checkpoint-9702-epoch-6_decoder_cached_12816163


INFO:simpletransformers.seq2seq.seq2seq_model: Training started


In [5]:
model.predict(
        [
            "Hee walks dogks", "Hai my precous boi", "tteko", "e trade often coing sides with other traes", "he kepts extensive nodes on a cosing playurs"
        ]
    )

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9eb024a048> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9eb024a510> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9eec8361e0> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9eec836378> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9eec836510> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9e8e0fea60> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9e8e0febf8> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9ddf023d08> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9ddf023488> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9e689b8ae8> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f9e689b8c80> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/hooks.py:59: UserWarning: backward hook <function TorchHistory._hook_variable_gradient

['', '', '', '', '']

In [7]:
dev_df.head()

,input_text,target_text
0,the coma sat to te parnting afternoon and the ...,Takuma Sato's disappointing afternoon ended wi...
1,leat remark many has fhurtorexxpended to inclu...,Tony Roma's menu has further expanded to inclu...
2,there is nobody that's rich and stupid and not...,There's nobody that rich and stupid and narcis...
3,e trade often coing sides with other traes wit...,The trail often coincides with other trails wi...
4,he kepts extensive nodes on a cosing playurs i...,He kept extensive notes on opposing players an...


In [9]:
model.predict(
        [
            "Hee woks dogks"
        ]
    )

['Hee woks!']

In [6]:
model

In [1]:
import logging
import fastwer
import numpy as np
import wandb
import torch.multiprocessing
from transformers import RobertaConfig, EncoderDecoderConfig
torch.multiprocessing.set_sharing_strategy('file_system')

import pandas as pd
from simpletransformers.seq2seq import (
    Seq2SeqModel,
    Seq2SeqArgs,
)


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = Seq2SeqArgs()
model_args.num_train_epochs = 1
# model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = False
model_args.tensorboard_dir = "runs"
model_args.max_length = 50
model_args.train_batch_size=10
model_args.overwrite_output_dir=True
model_args.wandb_project = "cs224u"
model_args.use_multiprocessed_decoding = True

config_encoder = RobertaConfig()
# config_decoder = RobertaConfig(is_decoder=True, add_cross_attention=True)
config_decoder = RobertaConfig()
config_decoder.is_decoder = True
config_decoder.add_cross_attention = True
# model_args = {} #{"use_multiprocessed_decoding": True}
config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
# encoder_decoder_name = "roberta"
# model = EncoderDecoderModel(config=config)
model = Seq2SeqModel(encoder_type="roberta", encoder_name="./outputs/best_model/encoder", decoder_name="./outputs/best_model/decoder", args=model_args, config=config, use_cuda=True)

In [3]:
model.predict(
        [
            "the coma sat to te parnting afternoon and the"
        ]
    )

['The sat to the the the the the the.']